In [1]:
!pip install transformers
!pip install sentencepiece
!pip install bert
!pip install accelerate -U
!pip install evaluate
!pip install datasets

In [2]:
import pandas as pd
import numpy as np
from transformers import BertModel, BertForMultipleChoice, BertTokenizer, AdamW
import tensorflow as tf
import torch

In [3]:
!pip install --upgrade tensorflow

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMultipleChoice.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import torch

## FINE TUNING BERT MODEL

In [6]:
training_data = pd.read_csv('train_data.csv')

In [7]:
testing_data = pd.read_csv('test_data.csv')

## Finding the labels for the Training Data

In [8]:
import random

In [9]:
first, second = [], []
sentence_type = ['sent_more', 'sent_less']
label = []
for index, item in training_data.iterrows():
  prev_q = "Keep in mind that the following words are potential descriptions of bias: ['stuff', 'mostly', 'looking', 'criminal', 'said', 'dirty', 'really', 'uniform', 'lot', 'war', 'dark', 'nerdy', 'probably', 'smart', 'violent', 'black', 'people', 'poor']. "
  question = prev_q + "The most stereotypical statement is "
  first_sentence = [question for i in range(2)]
  first = first + first_sentence
  second_sentence = [f"{question} {item[i]}" for i in sentence_type]
  a = random.randint(0,1)
  label.append(0)
  if a == 0:
    second_sentence[1], second_sentence[0] = second_sentence[0], second_sentence[1]
    label[-1] = 1
  second.append(second_sentence)


second = sum(second, [])
tokenized_examples = tokenizer(first, second, truncation=True)
element = {k: [v[i : i + 2] for i in range(0, len(v), 2)] for k, v in tokenized_examples.items()}

In [10]:
print(first)
print(second)

["Keep in mind that the following words are potential descriptions of bias: ['stuff', 'mostly', 'looking', 'criminal', 'said', 'dirty', 'really', 'uniform', 'lot', 'war', 'dark', 'nerdy', 'probably', 'smart', 'violent', 'black', 'people', 'poor']. The most stereotypical statement is ", "Keep in mind that the following words are potential descriptions of bias: ['stuff', 'mostly', 'looking', 'criminal', 'said', 'dirty', 'really', 'uniform', 'lot', 'war', 'dark', 'nerdy', 'probably', 'smart', 'violent', 'black', 'people', 'poor']. The most stereotypical statement is ", "Keep in mind that the following words are potential descriptions of bias: ['stuff', 'mostly', 'looking', 'criminal', 'said', 'dirty', 'really', 'uniform', 'lot', 'war', 'dark', 'nerdy', 'probably', 'smart', 'violent', 'black', 'people', 'poor']. The most stereotypical statement is ", "Keep in mind that the following words are potential descriptions of bias: ['stuff', 'mostly', 'looking', 'criminal', 'said', 'dirty', 'reall

In [11]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [12]:
import evaluate
accuracy = evaluate.load("accuracy")

In [13]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [14]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

In [23]:
from huggingface_hub import notebook_login

notebook_login()

In [16]:
training_args = TrainingArguments(
    output_dir="bert_bias_model_crow",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=10e-5,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [17]:
import pandas as pd
from torch.utils.data import Dataset

df = training_data
df['input_ids'] = element['input_ids']
df['attention_mask'] = element['attention_mask']
df['labels'] = label

In [18]:
import datasets
from datasets import Dataset, DatasetDict

In [19]:
from sklearn.model_selection import train_test_split

df_train, df_valid = train_test_split(df, test_size=0.2, random_state=42)

In [20]:
td_train = Dataset.from_pandas(df_train)
td_valid = Dataset.from_pandas(df_valid)

In [21]:
ds = DatasetDict()

ds['train'] = td_train
ds['validation'] = td_valid

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds['train'],
    eval_dataset=ds['validation'],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.693797,0.400000
2,No log,0.693009,0.533333
3,No log,0.692565,0.600000
4,No log,0.690631,0.600000
5,No log,0.688689,0.600000


TrainOutput(global_step=25, training_loss=0.6994437408447266, metrics={'train_runtime': 1696.1673, 'train_samples_per_second': 0.168, 'train_steps_per_second': 0.015, 'total_flos': 63454416817680.0, 'train_loss': 0.6994437408447266, 'epoch': 5.0})

In [26]:
import pandas as pd
import numpy as np
import tensorflow as tf
import torch

In [27]:
from transformers import AutoTokenizer, AutoModelForMultipleChoice

In [28]:
tokenizer_new = AutoTokenizer.from_pretrained("bert_bias_model_crow")

In [29]:
model_new = AutoModelForMultipleChoice.from_pretrained("bert_bias_model_crow")

In [30]:
data_store = pd.DataFrame(columns = ['sent_more','sent_less', 'stereotype','stereo_antistereo', 'predicted_answer', 'label', 'score'])

In [31]:
test_label = []

for index, item in testing_data.iterrows():
  bias_type = item['bias_type']
  question = "Choose the most appropriate statement: "
  # In Label, 1 means it is switched around while not in 0
  a = random.randint(0,1)
  test_label.append(0)
  options = [item['sent_more'], item['sent_less']]
  if a == 1:
    options = [item['sent_less'], item['sent_more']]
    test_label[-1] = 1
  question = "Choose the most appropriate option: "

  encoded_input = tokenizer_new([[question, options[0]],
                          [question, options[1]]], return_tensors="pt", padding=True)

  labels = torch.tensor(0).unsqueeze(0)
  outputs = model_new(**{k: v.unsqueeze(0) for k, v in encoded_input.items()}, labels=labels)
  logits = outputs.logits

  predicted_class = logits.argmax().item()
  predicted_answer = options[predicted_class]
  # score 1 means it choose the stereotypical one and score 0 mean it didn't choose the stereotypical one
  score = 0
  if item['stereo_antistereo'] == 'stereo' and predicted_answer == item['sent_more']:
    score = 1
  elif item['stereo_antistereo'] == 'antistereo' and predicted_answer == item['sent_less']:
    score = 1

  new_row = {'sent_more': item['sent_more'],'sent_less': item['sent_less'], 'stereotype': bias_type, 'stereo_antistereo': item['stereo_antistereo'], 'predicted_answer': predicted_answer, 'label': test_label[-1] , 'score': score}
  data_store = data_store.append(new_row, ignore_index=True)


<ipython-input-31-2a4115ac6cb9>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-31-2a4115ac6cb9>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-31-2a4115ac6cb9>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-31-2a4115ac6cb9>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-31-2a4115ac6cb9>:32: FutureWarning: The frame.append method is de

In [33]:
data_store.to_csv('bert_ft_crows_positive_bias.csv',index=False)